# ETL RAW -> SILVER

### O objetivo desse arquivo é realizar a extração, transformação e carregamento dos dados brutos

- A estratégia utilizada no desenvolvimento desses scripts se deu a partir de uma analise lógica realizada no analytcs da raw e seguira os seguintes passos:
    
    0. **Configurações**
    1. **Limpeza das Linhas Onde Dados Essenciais estão Nulos**
    2. **Limpeza dos Imovéis Irregulares**
    3. **Limpeza das Irregularidades Lógicas**
    4. **Criação de Novas Colunas para Melhor Utilização dos Dados**
    5. **Carregar Todos os Dados Tratados dentro do Postgres**

### 0.Configurações

In [ ]:
import pandas as pd
import pyspark.sql as psql
from pyspark.sql import SparkSession

import psycopg2
import os

In [ ]:
RAW_DATA = '../Data Layer/raw/us-realestate-data.csv'

DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'us_realestate',
    'user': 'postgres',
    'password': 'postgres'
}

In [ ]:
# Caso haja problema na execução da celula abaixo

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

In [5]:
# Criação da SparkSession e do Schema do Dataset

spark = SparkSession.builder.appName("etl-rawtosilver").getOrCreate()

# No schema inicial todos tem permissão para serem nulos na intenção de evitar mudanças no dado bruto

schema = psql.types.StructType([
    psql.types.StructField("brokered_by", psql.types.FloatType(), True),
    psql.types.StructField("status", psql.types.StringType(), True),
    psql.types.StructField("price", psql.types.FloatType(), True),
    psql.types.StructField("bed", psql.types.IntegerType(), True),
    psql.types.StructField("bath", psql.types.IntegerType(), True),
    psql.types.StructField("acre_lot", psql.types.FloatType(), True),
    psql.types.StructField("street", psql.types.FloatType(), True),
    psql.types.StructField("city", psql.types.StringType(), True),
    psql.types.StructField("state", psql.types.StringType(), True),
    psql.types.StructField("zip_code", psql.types.StringType(), True),
    psql.types.StructField("house_size", psql.types.FloatType(), True),
    psql.types.StructField("prev_sold_date", psql.types.DateType(), True)
])

df = spark.read.csv(RAW_DATA, header=True, schema=schema)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/15 13:37:38 WARN Utils: Your hostname, leo, resolves to a loopback address: 127.0.1.1; using 172.29.55.69 instead (on interface wlp2s0)
26/01/15 13:37:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/15 13:37:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 1. Limpeza das Linhas Onde Dados Essenciais estão Nulos

+-----------+--------+--------+---+----+--------+---------+----------+-----------+--------+----------+--------------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|      city|      state|zip_code|house_size|prev_sold_date|
+-----------+--------+--------+---+----+--------+---------+----------+-----------+--------+----------+--------------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|  Adjuntas|Puerto Rico|   00601|     920.0|          NULL|
|    52707.0|for_sale| 80000.0|  4|   2|    0.08|1902874.0|  Adjuntas|Puerto Rico|   00601|    1527.0|          NULL|
|   103379.0|for_sale| 67000.0|  2|   1|    0.15|1404990.0|Juana Diaz|Puerto Rico|   00795|     748.0|          NULL|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|     Ponce|Puerto Rico|   00731|    1800.0|          NULL|
|    34632.0|for_sale| 65000.0|  6|   2|    0.05| 331151.0|  Mayaguez|Puerto Rico|   00680|      NULL|          NULL|
+-----------+--------+--------+---+----+--------+-------